### 参考  
https://notebooks.githubusercontent.com/view/ipynb?browser=unknown_browser&color_mode=auto&commit=b34f8964fe3cdbbf0dd527dc381926721538ac8c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6d617473756f6c61622d6564752f646c3475732f623334663839363466653363646262663064643532376463333831393236373231353338616338632f6c6573736f6e342f6c6573736f6e345f736563325f65786572636973652e6970796e62&logged_in=false&nwo=matsuolab-edu%2Fdl4us&path=lesson4%2Flesson4_sec2_exercise.ipynb&platform=unknown_platform&repository_id=179020903&repository_type=Repository&version=0

### 実装①
LSTMを使ったSeq2Seqモデルで 文章→ぺこらポエム 変換を行う。  
使用するデータセット、train.sentenceとtrain.poemの中身は次のようになっています.  
train.sentenceの中身 (ポエムの要約)

凄いと噂のIDサバも見たいし、今年もあるんだー。あと、夏祭りの予習もしないと！（笑  
︙

train.poemの中身(要約前のポエム)

今日は久しぶりのマイクラ❣  凄いと噂のID鯖も見てみたいし 今年もあります！夏祭りの下準備もしなければ！  欲張りぺこーらぺこおおおおおお✨✨  
︙

### データの用意
まずはデータの読み込みです。
読み込む際、文頭を表す仮想単語（BOS, Beginning Of Sentence）として\<s>、文末を表す仮想単語（EOS, End Of Sentence）として<\s>を付加します。
また、BOS, EOSをつけた文章について、Tokenizerによって数値化を行います。
最後に、バッチ処理のため、各系列の長さをそろえておきます。これにはkeras.preprocessing.sequence.pad_sequencesを用います。

In [68]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_data(file_path):
    tokenizer = Tokenizer(filters="")
    whole_texts = []
    for line in open(file_path, encoding='utf-8'):
        whole_texts.append("<s> " + line.strip() + " </s>")
        
    tokenizer.fit_on_texts(whole_texts)
    
    return tokenizer.texts_to_sequences(whole_texts), tokenizer

# 読み込み＆Tokenizerによる数値化
x_train, tokenizer_en = load_data('./data/train.sentence.txt')
y_train, tokenizer_ja = load_data('./data/train.poem.txt')

en_vocab_size = len(tokenizer_en.word_index) + 1
ja_vocab_size = len(tokenizer_ja.word_index) + 1

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# パディング
x_train = pad_sequences(x_train, padding='post')
y_train = pad_sequences(y_train, padding='post')

seqX_len = len(x_train[0])
seqY_len = len(y_train[0])

### モデル構築
ここでは、LSTMを使用してSeq2Seqモデルを構築します。  
Embeddingレイヤーではmask_zero=Trueを引数として指定することで、計算上先程のパディング部分を無視するようにしています。  
また、Recurrentレイヤーに対するreturn_state=Trueやreturn_sequences=Trueの指定をLSTMレイヤーの生成時に行っています。  
なお、Functional APIによるモデル構築であることに注意してください。  

In [69]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM

emb_dim = 256
hid_dim = 256

## 符号化器
# Inputレイヤー（返り値としてテンソルを受け取る）
encoder_inputs = Input(shape=(seqX_len,))

# モデルの層構成（手前の層の返り値テンソルを、次の接続したい層に別途引数として与える）
# InputレイヤーとEmbeddingレイヤーを接続（+Embeddingレイヤーのインスタンス化）
encoder_embedded = Embedding(en_vocab_size, emb_dim, mask_zero=True)(encoder_inputs) # shape: (seqX_len,)->(seqX_len, emb_dim)
# EmbeddingレイヤーとLSTMレイヤーを接続（+LSTMレイヤーのインスタンス化）
_, *encoder_states = LSTM(hid_dim, return_state=True)(encoder_embedded)  # shape: (seqX_len, emb_dim)->(hid_dim, )
# このLSTMレイヤーの出力に関しては下記に補足あり

In [70]:
## 復号化器
# Inputレイヤー（返り値としてテンソルを受け取る）
decoder_inputs = Input(shape=(seqY_len,))

# モデルの層構成（手前の層の返り値テンソルを、次の接続したい層に別途引数として与える）
# InputレイヤーとEmbeddingレイヤーを接続
decoder_embedding = Embedding(ja_vocab_size, emb_dim) # 後で参照したいので、レイヤー自体を変数化
decoder_embedded = decoder_embedding(decoder_inputs)  # shape: (seqY_len,)->(seqY_len, emb_dim)
# EmbeddingレイヤーとLSTMレイヤーを接続（encoder_statesを初期状態として指定）
decoder_lstm = LSTM(hid_dim, return_sequences=True, return_state=True) # 後で参照したいので、レイヤー自体を変数化
decoder_outputs, _, _ = decoder_lstm(decoder_embedded, initial_state=encoder_states) # shape: (seqY_len, emb_dim)->(seqY_len, hid_dim)
# LSTMレイヤーとDenseレイヤーを接続
decoder_dense = Dense(ja_vocab_size, activation='softmax') # 後で参照したいので、レイヤー自体を変数化
decoder_outputs = decoder_dense(decoder_outputs) # shape: (seqY_len, hid_dim)->(seqY_len, ja_vocab_size)

# モデル構築（入力は符号化器＆復号化器、出力は復号化器のみ）
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
# 今回は、sparse_categorical_crossentropy（正解ラベルとしてone_hot表現のベクトルでなく数値を受け取るcategorical_crossentropy）を使用

### モデルの学習
モデルの学習時には、教師データとして1時点先の単語を示すデータを入力します。(train_target)  
学習時にはDecoderの入力に教師データを用います。  

In [71]:
import numpy as np

train_target = np.hstack((y_train[:, 1:], np.zeros((len(y_train),1), dtype=np.int32)))

model.fit([x_train, y_train], np.expand_dims(train_target, -1), batch_size=2, epochs=20, verbose=2, validation_split=0.2)

Epoch 1/20
31/31 - 7s - loss: 3.9871 - val_loss: 3.6470 - 7s/epoch - 213ms/step
Epoch 2/20
31/31 - 1s - loss: 3.3459 - val_loss: 3.6959 - 1s/epoch - 35ms/step
Epoch 3/20
31/31 - 1s - loss: 3.1461 - val_loss: 3.9964 - 1s/epoch - 36ms/step
Epoch 4/20
31/31 - 1s - loss: 3.0424 - val_loss: 4.2559 - 1s/epoch - 35ms/step
Epoch 5/20
31/31 - 1s - loss: 2.9586 - val_loss: 4.2383 - 1s/epoch - 36ms/step
Epoch 6/20
31/31 - 1s - loss: 2.8808 - val_loss: 4.7319 - 1s/epoch - 35ms/step
Epoch 7/20
31/31 - 1s - loss: 2.7851 - val_loss: 4.5280 - 1s/epoch - 35ms/step
Epoch 8/20
31/31 - 1s - loss: 2.6976 - val_loss: 5.4353 - 1s/epoch - 36ms/step
Epoch 9/20
31/31 - 1s - loss: 2.6166 - val_loss: 5.5986 - 1s/epoch - 35ms/step
Epoch 10/20
31/31 - 1s - loss: 2.5466 - val_loss: 5.7276 - 1s/epoch - 36ms/step
Epoch 11/20
31/31 - 1s - loss: 2.4453 - val_loss: 5.9013 - 1s/epoch - 35ms/step
Epoch 12/20
31/31 - 1s - loss: 2.3765 - val_loss: 6.4508 - 1s/epoch - 37ms/step
Epoch 13/20
31/31 - 1s - loss: 2.3026 - val_loss

### モデルによる生成
先程学習したモデルを使用して、系列を生成してみましょう。  
そのためにまずは学習したモデルを組み込んだ、系列生成用のモデルを構築します。  
学習時との違いは、復号化器が1ステップずつ実行できるよう、状態ベクトルの入力と出力をモデルの定義に加えている点です。  
(また、1ステップ前の状態を引き継いで生成が可能になるように、復号化器のモデルの初期状態を指定可能にしています。)  
生成する際のDecoderの入力には翻訳先の教師データは用いません。  

In [72]:
# サンプリング用（生成用）のモデルを作成

# 符号化器（学習時と同じ構成、学習したレイヤーを利用）
encoder_model = Model(encoder_inputs, encoder_states)

# 復号化器
decoder_states_inputs = [Input(shape=(hid_dim,)), Input(shape=(hid_dim,))] # decorder_lstmの初期状態指定用(h_t, c_t)

decoder_inputs = Input(shape=(1,))
decoder_embedded = decoder_embedding(decoder_inputs) # 学習済みEmbeddingレイヤーを利用
decoder_outputs, *decoder_states = decoder_lstm(decoder_embedded, initial_state=decoder_states_inputs) # 学習済みLSTMレイヤーを利用
decoder_outputs = decoder_dense(decoder_outputs) # 学習済みDenseレイヤーを利用

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

### モデルを使用した生成（予測）
生成では、未知のデータに対してモデルを適用するので正解ラベルはわかりません。  
そこで、代わりに前のステップで予測した単語を各ステップでの入力とします。  
そして, 系列の終わりを表す単語 (\</s>) が出力されるまで繰り返します。（最初の入力は\<s>を使用します）  

In [73]:
def decode_sequence(input_seq, bos_eos, max_output_length = 1000):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.array(bos_eos[0])  # bos_eos[0]="<s>"に対応するインデックス
    output_seq= bos_eos[0][:]
    
    while True:
        output_tokens, *states_value = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = [np.argmax(output_tokens[0, -1, :])]
        output_seq += sampled_token_index
        
        if (sampled_token_index == bos_eos[1] or len(output_seq) > max_output_length):
            break

        target_seq = np.array(sampled_token_index)

    return output_seq

In [74]:
detokenizer_en = dict(map(reversed, tokenizer_en.word_index.items()))
detokenizer_ja = dict(map(reversed, tokenizer_ja.word_index.items()))

text_no = 8
input_seq = pad_sequences([x_test[text_no]], seqX_len, padding='post')
bos_eos = tokenizer_ja.texts_to_sequences(["<s>", "</s>"])

print('元の文:', ' '.join([detokenizer_en[i] for i in x_test[text_no]]))
print('生成文:', ' '.join([detokenizer_ja[i] for i in decode_sequence(input_seq, bos_eos)]))
print('正解文:', ' '.join([detokenizer_ja[i] for i in y_test[text_no]]))

元の文: <s> ペコラはロケランで何でも破壊してしまうパワフルな女の子です。彼女は仲間になる人を探している。報酬はすべて彼女のものです。 </s>
1/1 [==============================] - 0s 21ms/step
生成文: <s> 前回苦しめられたウォーデン… 伝説の料理「龍の髭」を!!!!!! 1本…2本…4本…8本…16本… 最後には1万6千本以上の髭を作るぺこ💪✨ 新人龍の髭職人ぺこーらの技をご覧あれ！ </s>
正解文: <s> げきうま！げきうま！やばいぐらい物資がうますぎるらしい これはいくしかないぺこだよね？ 危険？危ない？お前じゃ無理だ。 おいおい、何言ってんだい。 ぺこーらの右腕みえねえのか。 このロケランですべてを破壊してやるよ。 ともについてくるやつがいるならついてこい！ 報酬はすべてぺこらのもの </s>
